In [2]:
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma
from valentine.algorithms import Cupid
import os
import multiprocessing as mp
from multiprocessing import Pool
import time
import pandas as pd
from compute_match import *

# Preparazione data frames

In [3]:
csv_path = './csv/'

## Creazione dizionari che associa il nome del file al file .json

In [3]:
# dataset sono in formato json necessariamente
path = "../../json_datasets"

# Dizionario Key=Nome del file, Value=File json associato (letto da filepath)
data_frames = {}
# Per tutti i file dentro la cartella dei dataset
for file in os.listdir(path):
    # Prendo il path del file
    filepath = f"{path}/{file}"
    # Update del dizionario
    data_frames[file] = (pd.read_json(filepath))

## Creazione di tutte le coppie di dataset da valutare (evitando ripetizioni)

In [4]:
# Prendo tutti i nomi dei file
keys = list(data_frames.keys())
tuples = []
# Il numero di dataframes
dfs_len = len(keys)

In [35]:
# Preso un dataframe, lo confronto con tutti i dataframes successivi (così evito di confrontare coppie già confrontate)
for i in range(dfs_len):
    for j in range(i + 1, dfs_len):
        dfl = data_frames[keys[i]]
        dfr = data_frames[keys[j]]
        # le tuple da passare al metodo di matching = nome dataset1, nome dataset2, dataframe1, dataframe2
        tuples.append((keys[i], keys[j], dfl, dfr))

# Valentine schema matching

In [ ]:
def calculate_match_multithread(tuples):
    result = []
    # Creo un pool per l'esecuzione multi processore (per velocizzare il matching tra tutte le coppie di dataframes)
    with Pool() as pool:
        # Ogni pool esegue la funzione (primo parametro), passandogli le tuple (secondo parametro)
        # result è una lista di tuple ove una tupla è il risultato della funzione mappata
        result = pool.map(calculate_match_coma_schema, tuples)
    return result

def matches_to_table(result):
    matches = pd.DataFrame()
    # Per ogni risultato ottenuto dal matching
    for match in result:
    # Prendo il valore ritornato dalla funzione valentine_match 
    # e.g. match[2] = ((table_1, 'Cited by'),(table_2, 'Cited by')): 0.83
        for key in match[2].keys():
            # e.g. key = (table_1, 'Cited by'),(table_2, 'Cited by')
            d = dict()
            d["table_1"] = match[0]
            d["table_2"] = match[1]
            d["column_table_1"] = key[0][1]
            d["column_table_2"] = key[1][1]
            d["match_value"] = match[2][key]
            # Aggiungiamo al dataframe l'ennupla
            matches = pd.concat([matches, pd.DataFrame(d, index=[0])], ignore_index=True)
    return matches

# Tabella per la visualizzazione dei risultati dello schema matching con Valentine

In [ ]:
start_time = time.perf_counter()
# Calcola i match per tutte le coppie di datasets definite in tuples
result = calculate_match_multithread(tuples)
finish_time = time.perf_counter()

# Definizione del nome delle colonne della tabella
columns = ["table_1", "table_2", "column_table_1", "column_table_2", "match_value"]
# Creo il dataframe (la tabella in questione)
matches = pd.DataFrame(columns=columns)

# Porta tutti i match in formato tabellare
matches = matches_to_table(result)

In [ ]:
pd.DataFrame(matches).to_csv(csv_path + "matches.csv")

# Schema mediato

In [6]:
# Schema mediato definito manualmente tramite attributi di interesse e che fanno maggior match
mediated_schema_columns = ('name', 'country', 'market cap', 'founded year', 'employees', 'industry', 'sector',
     'ceo', 'revenue', 'Stock', 'share price', 'city', 'address', 'website')


In [ ]:

mediated_schema = pd.DataFrame(columns=mediated_schema_columns)

In [ ]:
# Creazione di tutte le coppie (schema-mediato, dataset)
matches_with_mediated = {}
for i in range(dfs_len):
    dfl = mediated_schema
    dfr = data_frames[keys[i]]
    # le tuple da passare al metodo di matching = nome dataset1, nome dataset2, dataframe1, dataframe2
    tuples.append(('mediated_schema', keys[i], dfl, dfr))

In [ ]:
# Matches con lo schema mediato
result = calculate_match_multithread(tuples)

In [1]:
matches = matches_to_table(result)


NameError: name 'matches_to_table' is not defined

In [69]:
pd.DataFrame(matches).to_csv(csv_path + "matches_with_mediated.csv")

# Riempimento schema mediato

In [8]:
matches = pd.read_csv(csv_path + 'matches_with_mediated.csv')
tables_to_columns = dict()
# Per tutte le tabelle in match nella colonna table 2 vado a prendermi
# le colonne con match >= 0.5 con una colonna dello schema mediato
for table_name in set(matches['table_2']):
    # Prende le colonne con table 2 uguale a table name
    table = matches.loc[matches['table_2'] == table_name]
    columns = set()
    for ind in table.index:
        if table['match_value'][ind] >= 0.5:
            columns.add((table['column_table_1'][ind], table['column_table_2'][ind]))
    tables_to_columns[table_name] = columns

In [52]:
schema_final = pd.DataFrame(columns=mediated_schema_columns)

# Per tutti i dataset vai a inserire i dati nello schema mediato
for df_name in tables_to_columns.keys():
    df_aligned = pd.DataFrame()
    
    # Per tutti i match del dataset effettua una proiezione sulle colonne in match e rinominale per allinearle allo schema mediato
    for match in tables_to_columns[df_name]:    
        tmp = data_frames[df_name][match[1]].to_frame()
        tmp = tmp.rename(columns={match[1]: match[0]})
        df_aligned = pd.concat([df_aligned, tmp], axis=1)

    try:
        schema_final = pd.concat([schema_final, df_aligned], ignore_index=True, sort=False)
    except:
        print("Cannot concat schema final with: ", df_name)

Cannot concat schema final with:  FR-ft.json
Cannot concat schema final with:  DDD-ft.com.json
Cannot concat schema final with:  Silvestri-ft.com.json
Cannot concat schema final with:  Gren-ft.com.json


In [46]:
schema_final = pd.read_csv(csv_path + "schema_final.csv", index_col=0)
schema_final = schema_final.reset_index(drop=True)
# schema_final.astype({"id": Int64})
# idx = {"id": list(schema_final.index)}
# schema_final = pd.concat([pd.DataFrame(idx), schema_final], axis=1)
# schema_final['id'] = schema_final['id'].astype('Int64')
print(schema_final)

                                            name country market cap  \
0                             Berkshire Hathaway     NaN        NaN   
1                                           ICBC     NaN        NaN   
2       Saudi Arabian Oil Company (Saudi Aramco)     NaN        NaN   
3                                 JPMorgan Chase     NaN        NaN   
4                        China Construction Bank     NaN        NaN   
...                                          ...     ...        ...   
181405                             Daily Journal     NaN    $0.40 B   
181406                                    Ouster     NaN    $0.22 B   
181407                                    Ouster     NaN    $0.22 B   
181408                                    Ouster     NaN    $0.22 B   
181409                      Poseida Therapeutics     NaN    $0.60 B   

       founded year employees                        industry sector  ceo  \
0            1939.0       NaN          Diversified Financials    NaN  

In [47]:
schema_final.to_csv(csv_path + "schema_final.csv", index_label='id')

In [ ]:
sch = schema_final.iloc[:10]
sch.to_csv(csv_path + "schema_final_10.csv", index_label='id')